In [ ]:
import numpy as np
from tensorflow.keras import layers, models, optimizers, metrics
from matplotlib import pyplot as plt
import pandas as pd
import tensorflow as tf

In [ ]:
normal = pd.read_csv("../ecg/ptbdb_normal.csv")
abnormal = pd.read_csv("../ecg/ptbdb_abnormal.csv")

In [ ]:
normal = normal.values[:,:-1]
abnormal = abnormal.values[:,:-1]

In [ ]:
def show_ecg(signal):
    plt.figure(figsize=(12, 1))
    plt.plot(np.arange(signal.size), signal, c="black")
    plt.show()

In [ ]:
show_ecg(normal[10])

In [ ]:
show_ecg(abnormal[10])

In [ ]:
normal.shape, abnormal.shape

In [ ]:
X = np.concatenate((normal, abnormal))
y = np.concatenate((np.zeros(normal.shape[0]), np.ones(abnormal.shape[0])))

In [ ]:
X = np.expand_dims(X, axis=2)

In [ ]:
X.shape, y.shape

In [ ]:
p = np.random.permutation(y.size - 1)
X = X[p]
y = y[p]

In [ ]:
embed_size = 2048
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
embedded = tsne.fit_transform(X[:embed_size])
plt.figure(figsize=(8, 4))
plt.scatter(embedded[:,0], embedded[:,1], c=(1 - y[:embed_size]), cmap='bwr', s=3)
plt.show()

In [ ]:
split = int(y.size * 0.7)
X_train = X[0:split]
y_train = y[0:split]
X_test = X[split:]
y_test = y[split:]

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape 

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.Conv1D(32, (3), activation='relu', padding='same'))
    model.add(layers.Conv1D(64, (3), activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=(2)))
    model.add(layers.Dropout(0.25))

    model.add(layers.Conv1D(128, (3), activation='relu', padding='same'))
    model.add(layers.Conv1D(256, (3), activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=(2)))
    model.add(layers.Dropout(0.25))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation="sigmoid"))

    model.build(input_shape = (None, X.shape[1], 1))
    model.summary()
    return model

In [ ]:
model = build_model()

In [ ]:
#optimizer = optimizers.Adadelta(learning_rate=1.0, rho=0.95, epsilon=1e-06)

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=["accuracy" ],
)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), shuffle=True, epochs=10)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred[y_pred > 0.5] = 1
y_pred[y_pred < 0.5] = 0

In [ ]:
embed_size = 2048
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
embedded = tsne.fit_transform(X_test[:embed_size].reshape(embed_size, -1))

In [ ]:
plt.figure(figsize=(8, 4))
plt.scatter(embedded[:,0], embedded[:,1], c=(1 - y_pred[:embed_size]), cmap='bwr', s=3)
plt.show()
y_test_ex = np.expand_dims(y_test[:embed_size], 1)
plt.figure(figsize=(8, 4))
plt.scatter(embedded[:,0], embedded[:,1], c=(1 - (y_pred[:embed_size] - y_test_ex[:embed_size])), cmap='bwr', s=3)
plt.show()

In [ ]:
con_mat = tf.math.confusion_matrix(labels=y_test, predictions=y_pred).numpy()

In [ ]:
fig, ax = plt.subplots()
ax.matshow(con_mat, cmap='bwr')
for (i, j), z in np.ndenumerate(con_mat):
    ax.text(j, i, '{:0}'.format(z), ha='center', va='center', color="white", fontsize=20)